# Iteration 0: spot to cell assignemt

### This is a notebook demonstrating the calculations during the spot to cell assignment step in pciSeq

In [12]:
!pip install git+https://github.com/acycliq/pciSeq.git@dev_3d

  Cloning https://github.com/acycliq/pciSeq.git (to revision dev_3d) to /tmp/pip-req-build-bjrxvwj1
  Running command git clone --filter=blob:none --quiet https://github.com/acycliq/pciSeq.git /tmp/pip-req-build-bjrxvwj1
  Running command git checkout -b dev_3d --track origin/dev_3d
  Switched to a new branch 'dev_3d'
  Branch 'dev_3d' set up to track remote branch 'dev_3d' from 'origin'.
  Resolved https://github.com/acycliq/pciSeq.git to commit 31fd7688f1f848d9e59931636ce39479614ea23d
  Preparing metadata (setup.py) ... done


In [13]:
import pandas as pd
import numpy as np
import scipy
from scipy.special import softmax
import numpy_groupies as npg
import gdown
import pciSeq

In [14]:
file_id = "1wY1PUvAwk_S9MhKbCyB0HzH9oxNloP0M"
url = f"https://drive.google.com/uc?export=download&id={file_id}"
output = "pciSeq.pickle"

In [15]:
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1wY1PUvAwk_S9MhKbCyB0HzH9oxNloP0M
From (redirected): https://drive.google.com/uc?export=download&id=1wY1PUvAwk_S9MhKbCyB0HzH9oxNloP0M&confirm=t&uuid=e181ed19-8bb2-49cd-8e1b-7cb5c4d127bb
To: /content/pciSeq.pickle
100%|██████████| 1.79G/1.79G [00:16<00:00, 109MB/s]


'pciSeq.pickle'

In [16]:
obj = pd.read_pickle(output)

In [17]:
# Set the hyperparameters
rSpot = 2.0               # Spread of the negative binomial
inefficiency = 0.2        # multiplicative factor applied to the single cell data
InsideCellBonus = 0       # No inside cell bonus
misread_density = 1.e-06  # amount of noise in the data

In [18]:
# The label of the cell under investigation
my_cell = 9382

### Preliminaries
Define some functions that we will use in the remainder

In [19]:
def logGammaExpectation(alpha_param, lambda_param):
    """
    Calculates the log expectation of a gamma distribution.
    Assumes the shape - rate parameterisation of the gamma density
    """
    a = alpha_param[:, :, None] # add a new dimension of size 1 to the end of the array, to enable broadcasting
    return scipy.special.psi(a) - np.log(lambda_param)

In [20]:
def geneCount(obj, spots_parent_cell_prob):
    """
    Build expected gene counts per cell from spot probabilities.

    spots_parent_cell_prob is an array of shape num_spots-num_neighbours
    where the value at position [i,j] expresses the probability that the
    spot at row i is assigned to the j-th neighbouring cell

    Returns:
        out: 2D array shape (nC, nG)
            - Row 0 is zeros (we exclude background here)
            - Rows 1.. are the expected counts for each cell
        background_counts: 1D array length nG
            - Expected counts for background (spots not assigned to any cell)
    """
    # make an array nS-by-nN and fill it with the spots id
    gene_ids = np.tile(obj.spots.gene_id, (obj.nN, 1)).T

    # Flatten gene_ids to a long list of length n_spots * nN
    gene_ids = gene_ids.ravel()

    # Flatten the array that holds the spots -> cell probabilities
    # Then flatten the array that holds the spots -> cell label
    probs = spots_parent_cell_prob.ravel()
    cell_ids = obj.spots.parent_cell_id.ravel()

    # Create an index array for grouping: first row = cell, second row = gene
    group_idx = np.vstack((cell_ids, gene_ids))

    # 5) Sum probabilities for each (cell, gene) pair
    #    N_cg[c, g] = total expected count of gene g in cell c
    N_cg = npg.aggregate(group_idx, probs, func='sum', size=(obj.nC, obj.nG))

    # 6) Prepare output array (exclude background row)
    out = np.zeros((obj.nC, obj.nG), dtype=np.float32)
    out[1:, :] = N_cg[1:, :]

    # 7) Extract background counts from row 0 of N_cg
    background_counts = N_cg[0, :]

    return out, background_counts

In [21]:
def spots_to_cell(obj) -> np.ndarray:
    """
    calculates spot-to-cell assignment probabilities.

    Returns:
        parent_cell_prob: 2D array shape (num_spots, num_neighbours)
        Each row sums to 1 and gives probabilities over nearby cells + background.
    """
    # nN = number of nearby cells + 1 for misread background
    nN = obj.nN

    # nS = number of spots (data points)
    nS = obj.spots.data.gene_name.shape[0]

    # Initialize log-scores for each spot and each possibility (cells + background)
    wSpotCell = np.zeros([nS, nN], dtype=np.float64)

    # Get gene names for each spot (used to look up expected counts)
    gn = obj.spots.data.gene_name.values

    # expected_counts[i,j]: average (log)count of gene j in cell type of spot i
    # expected_counts will be an array of shape num_spots - num_classes
    expected_counts = obj.single_cell.log_mean_expression.loc[gn].values

    # Fill last column of wSpotCell with log-probability of misread
    wSpotCell[:, -1] = np.log(misread_density)

    # Loop over each of the first nN-1 nearest cells
    for n in range(nN - 1):
        # sn: array of cell IDs for the n-th closest cell to each spot
        sn = obj.spots.parent_cell_id[:, n]

        # cp: class probabilities for each spot's candidate cell
        cp = obj.cells.classProb[sn]

        # term_1: This is what I call attention.
        # np.einsum('sk, sk->s', expected_counts, cp) does:
        # For each spot s and class k, multiply expected_counts[s,k] * cp[s,k],
        # then sum over all classes k to get one number per spot.
        # In practice this means that when genes with high expected counts
        # are aligned with high cell class probs this term will be high
        term_1 = np.einsum('sk, sk -> s', expected_counts, cp)

        # term_2: combine class probabilities and log_gamma
        log_gamma = log_gamma_bar[obj.spots.parent_cell_id[:, n], obj.spots.gene_id]
        term_2 = np.einsum('sk, sk -> s', log_gamma, cp)

        # gaussian log-likelihood based on distance to cell center
        mvn_loglik = obj.spots.mvn_loglik(
            obj.spots.xyz_coords, sn, obj.cells, obj.config['is3D']
        )

        # Sum contributions into the log-score for this cell column
        wSpotCell[:, n] = term_1 + term_2 + mvn_loglik

    # Apply a bonus to spots inside cell boundaries.
    # In this exmaple InsideCellBonus = 0 and the line below can be ignored
    bonus_mask = obj.spots.bonus_mask * InsideCellBonus
    wSpotCell += bonus_mask

    # Convert log-scores to probabilities (softmax over each spot's row)
    parent_cell_prob = softmax(wSpotCell, axis=1)

    return parent_cell_prob


### Main

In [22]:
# Lets have a look at our data
obj.spots.data

,x,y,z,plane_id,label,gene_name,score
spot_id,,,,,,,
0,5239.818359,4314.244141,86.400108,26,2611,Abi3bp,0.5483
1,6201.818359,4442.244141,76.757248,23,0,Abi3bp,0.3079
2,6388.818359,4557.244141,189.257233,58,17811,Abi3bp,0.4617
3,5443.818359,4617.244141,67.114388,20,0,Abi3bp,0.7686
4,5830.818359,4667.244141,192.471527,59,0,Abi3bp,0.8580
...,...,...,...,...,...,...,...
889302,562.801392,2101.522461,151.480103,47,0,Zic1,0.3132
889303,561.801392,2119.522461,125.765816,39,0,Zic1,0.9270
889304,562.801392,2119.522461,132.194382,41,0,Zic1,0.9360


In the datafram above, the column with the name 'label' shows the label of the cell the spot lies within(ie the spot is inside the cell boundaries) otherwise it is set to 0

### 1. Gene counts
First initialise the cell gene counts

In [23]:
# Count how many times each gene appears in each cell label
ini_cgc = obj.spots.data.groupby(['label', 'gene_name']).size().reset_index(name='counts')
ini_cgc

,label,gene_name,counts
0,0,Abi3bp,82
1,0,Acly,967
2,0,Adcyap1,113
3,0,Adora2a,118
4,0,Afp,32
...,...,...,...
300634,18882,Pglyrp1,1
300635,18884,Mobp,1
300636,18884,Plp1,1
300637,18884,Stmn1,1


#### 1.1 Comment: We could have used the function geneCount defined above to calculate cgc.

In [24]:
# We just need the correct input array, which is shown below
ini_prob = obj.spots.ini_cellProb(obj.spots.parent_cell_id, obj.config)
ini_prob

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

The ini_prob array above is an array of shape num_spots - num_neighours.
This initial state is a binary array (only 0s and 1s). If a spot is drawn inside the
cell boundaries the corresponding value for the neigbbouring cell is 1.
For example the first row above is [0, 1, 0, 0, 0, 0, 0] which means that the first
spot is inside the boundaries of the 2nd closest cell.
Not that the last column is reserved for the background

In [25]:
# Feed now ini_prob to geneCount
cgc, _ = geneCount(obj, ini_prob)

In [26]:
# Lets check the gene counts for the cell we are intrested in.
df_temp = pd.DataFrame(cgc, columns=obj.genes.gene_panel)
df_temp.iloc[my_cell].sort_values(ascending=False)

,9382
Plp1,5.0
Hsd11b1,2.0
Maob,1.0
Rims4,1.0
Qk,1.0
...,...
Fosb,0.0
Fos,0.0
Fn1,0.0
Flt1,0.0


In [27]:
# Get the total gene counts for that cell
df_temp.iloc[my_cell].sum()

np.float32(10.0)

In [28]:
# Indeed those gene counts above look to be the same with those obtained with the groupby function
ini_cgc[ini_cgc.label == my_cell].sort_values(by='counts', ascending=False)

,label,gene_name,counts
147567,9382,Plp1,5
147565,9382,Hsd11b1,2
147566,9382,Maob,1
147568,9382,Qk,1
147569,9382,Rims4,1


### 2. Gene expressions are not constant across cells, even within the same cell type. We use a gamma distributed variable to model these flunctutations.

In [29]:
# First, get the count matrix from the single cell data
class_definitions = obj.single_cell.mean_expression

# scale by the inefficienct and add the hyperparamer r
_lambda = class_definitions * inefficiency + rSpot

# Add the hyperparameter r to the cell gene counts
_alpha = cgc + rSpot

In [30]:
# you can now calculate the expectation of the log(gamma)
log_gamma_bar = logGammaExpectation(_alpha, _lambda.values)
log_gamma_bar.shape

(18888, 314, 62)

### 3. Spot to cell probabilities

In [31]:
# call now the spots_to_cell function
spot_to_cell_prob = spots_to_cell(obj)
spot_to_cell_prob.shape

(889307, 7)

In [32]:
# Wrap it inside a dataframe and append the cell label and gene name at the end
df = pd.DataFrame(spot_to_cell_prob)
df = df.assign(label = obj.spots.data.label)
df = df.assign(gene_name = obj.spots.data.gene_name)
df

,0,1,2,3,4,5,6,label,gene_name
0,0.128056,1.334386e-01,1.277216e-02,4.563818e-03,2.861165e-03,7.076789e-05,0.718238,2611,Abi3bp
1,0.033328,1.271908e-03,6.281683e-09,1.534100e-11,5.502948e-12,1.582216e-12,0.965400,0,Abi3bp
2,0.431922,1.397126e-01,4.334395e-04,3.225576e-05,1.007416e-05,2.247918e-07,0.427890,17811,Abi3bp
3,0.000012,1.597641e-08,1.845006e-12,1.701594e-12,5.443440e-17,4.758765e-19,0.999988,0,Abi3bp
4,0.080357,1.741615e-06,4.591152e-07,1.243300e-07,7.779305e-09,5.384647e-09,0.919641,0,Abi3bp
...,...,...,...,...,...,...,...,...,...
889302,0.101678,3.756624e-03,2.887198e-03,1.943201e-03,1.838218e-03,3.049298e-04,0.887592,0,Zic1
889303,0.124858,9.334895e-02,3.912020e-02,9.090416e-04,8.581013e-05,6.115623e-05,0.741617,0,Zic1
889304,0.086455,6.549714e-02,4.904661e-02,6.650077e-04,2.168707e-04,1.271516e-04,0.797993,0,Zic1
889305,0.437119,2.458317e-04,4.882595e-06,3.686288e-06,2.154209e-06,1.309718e-06,0.562623,12318,Zic1


##### ***** Comment ****

Here's how to interpret the dataframe above:

For the spot in the first row, the gene read is 'Abi3bp'. This spot is located within the boundaries of cell '2611'.  The table shows the probabilities of this spot belonging to different cells.  There's a 12.8% probability that the spot is associated with the first closest cell, a 0.0133% probability for the second closest cell, and a 71% probability that the signal comes from background noise (i.e., it's a misread).

It's important to note that since the spot is inside cell '2611', it's likely that this is also its closest cell, but this isn't guaranteed. To find the labels of the neighboring cells, you would look up the obj.spots.parent_cell_id array.

In [33]:
obj.spots.parent_cell_id

array([[ 9528,  2611,  8030, ...,  4080, 10801,     0],
       [ 7386,  1436, 14889, ...,  9961, 16558,     0],
       [17811, 18276, 18275, ..., 15972, 12045,     0],
       ...,
       [11089,  7226, 11891, ..., 16168, 12725,     0],
       [12318, 11894,  4916, ...,  2985,  7559,     0],
       [17414, 11090, 16995, ..., 13581,  7226,     0]], dtype=uint32)

Indeed, the spot in the top row is closer to cell 9528 than to 2611. Lets doublecheck this.

The spot coords are x:5239.818359, y:4314.244141, z:86.400108 as we can see from the table below

In [34]:
obj.spots.data.head(1)

,x,y,z,plane_id,label,gene_name,score
spot_id,,,,,,,
0,5239.818359,4314.244141,86.400108,26,2611,Abi3bp,0.5483


The centroid of cell 2611 is:

In [35]:
obj.cells.centroid.iloc[2611]

,2611
x,5226.766113
y,4312.259277
z,56.518356


The centroid of cell 9528 is:

In [36]:
obj.cells.centroid.iloc[9528]

,9528
x,5256.026367
y,4331.091309
z,104.837448


In [37]:
# The squared distance of the spot to cell 2611
(5239.81-5226.76)**2 + (4314.24-4312.25)**2 + (86.40-56.51)**2

1067.6747000000043

In [38]:
# The squared distance of the spot to cell 9528
(5239.81-5256.02)**2 + (4314.24-4331.09)**2 + (86.40-104.83)**2

886.3515000000132

##### ***** End of Comment ****

In [39]:
# pass now the updated spot -> cell probabilities to get the latest cell gene counts
gene_counts, backround_counts = geneCount(obj, spot_to_cell_prob)

In [40]:
# It looks better in a dataframe
df = pd.DataFrame(gene_counts, columns=obj.genes.gene_panel)
df

,Abi3bp,Acly,Adcyap1,Adora2a,Afp,Agrp,Agt,Akr1c18,Aldh1a1,Aldoc,...,Tyrobp,Ucn,Ucn3,Vim,Vip,Vtn,Wfs1,Yjefn3,Zcchc12,Zic1
0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000e+00,0.000000e+00
1,0.0,5.389310e-06,0.0,0.0,0.0,0.000000e+00,0.000060,0.0,0.000000,6.373475e-03,...,0.000000e+00,0.0,0.0,0.000000,0.0,0.519070,0.04534,0.0,0.000000e+00,0.000000e+00
2,0.0,2.411444e-06,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00,...,1.026202e-03,0.0,0.0,0.062911,0.0,0.243665,0.00000,0.0,0.000000e+00,0.000000e+00
3,0.0,1.547951e-11,0.0,0.0,0.0,6.845368e-15,0.000000,0.0,0.000000,3.122204e-18,...,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000e+00,1.155959e-08
4,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000000,0.0,0.762437,0.00000,0.0,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18883,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000e+00,0.000000e+00
18884,0.0,5.076383e-11,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00,...,5.330432e-11,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,7.746456e-10,3.913582e-05
18885,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.027729,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000047,0.0,0.000000,0.00000,0.0,0.000000e+00,0.000000e+00
18886,0.0,3.547166e-03,0.0,0.0,0.0,0.000000e+00,0.000006,0.0,0.000000,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000002,0.0,0.000000,0.00000,0.0,1.745386e-06,8.574262e-02


In [41]:
# Lets now get the gene counts for cell 9382
df.iloc[my_cell].sort_values(ascending=False)

,9382
Plp1,4.034151
Hsd11b1,1.266736
Rims4,0.584342
Qk,0.538230
Cryab,0.452948
...,...
Tnnt1,0.000000
Tpbg,0.000000
Tpd52l1,0.000000
Tph2,0.000000


Those gene counts are exactly the same as those in the screenshot below:

![screenshot.png](https://github.com/acycliq/pciSeq_iter_0/blob/master/screenshot.png?raw=1)